## Нестандартные подходы
* "Навешивать" на модель дополнительные лоссы, то есть минимизировать одновременно несколько функций потерь.
* Обучать параллельно на нескольких датасетах и/или несколько моделей. Например, GAN
* Следить за тем, что происходит в процессе обучения 

https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing

In [25]:
from tensorflow.keras import Model, layers, Sequential
from tensorflow import keras
import tensorflow as tf
import numpy as np
from keras.metrics import sparse_categorical_accuracy
from itertools import permutations

In [14]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train = X_train.reshape(-1, 28*28)/255
X_test = X_test.reshape(-1, 28*28)/255

In [21]:
class CustomModel(Model):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.hidden_layers = [layers.Dense(784, 'relu') for _ in range(2)]
        self.head = layers.Dense(10, 'softmax')
    
    # def call(self, input, training=None, order=None):
    #     output = input
    #     order = order or np.random.choice(range(2), 2, replace=False)
    #     for layer_idx in order:
    #         output = self.hidden_layers[layer_idx](output)
    #     return self.head(output)
    
    def call_hidden_layers_in_order(self, input, order):
        # note
        # Если мы не передаём в .compile() параметры run_eagerly=True, 
        # то метод .call() вызывается только один раз и строит статический граф вычислений. 
        # Поэтому такой метод наоборот существенно ускорит обучение и инференс сети.
        output = input
        for layers_idx in order:
            output = self.hidden_layers[layers_idx](output)
        return output
    
    def call(self, input, training=None, order=None):
        if order:
            output = self.call_hidden_layers_in_order(input, order)
        else:
            all_possible_orders = list(permutations(range(len(self.hidden_layers)))) # self.hidden_layers_count
            all_outputs_dict = {order: self.call_hidden_layers_in_order(input, order) for order in all_possible_orders}
            all_outputs = tf.stack(list(all_outputs_dict.values()))
            index = tf.random.uniform((), minval=0, maxval=len(all_outputs), dtype=tf.int32)
            output = all_outputs[index]
        return self.head(output)

In [22]:
model = CustomModel()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics='accuracy',
              run_eagerly=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=1024)

Epoch 1/20
59/59 [==============================] - 22s 371ms/step - loss: 0.8498 - accuracy: 0.7854 - val_loss: 0.3140 - val_accuracy: 0.9087
Epoch 2/20
59/59 [==============================] - 23s 384ms/step - loss: 0.2731 - accuracy: 0.9180 - val_loss: 0.1953 - val_accuracy: 0.9408
Epoch 3/20
59/59 [==============================] - 20s 342ms/step - loss: 0.1708 - accuracy: 0.9491 - val_loss: 0.1513 - val_accuracy: 0.9548
Epoch 4/20
59/59 [==============================] - 20s 333ms/step - loss: 0.1338 - accuracy: 0.9606 - val_loss: 0.1309 - val_accuracy: 0.9590
Epoch 5/20
59/59 [==============================] - 19s 330ms/step - loss: 0.1079 - accuracy: 0.9679 - val_loss: 0.1150 - val_accuracy: 0.9662
Epoch 6/20
59/59 [==============================] - 20s 338ms/step - loss: 0.0937 - accuracy: 0.9721 - val_loss: 0.0997 - val_accuracy: 0.9694
Epoch 7/20
59/59 [==============================] - 20s 336ms/step - loss: 0.0746 - accuracy: 0.9783 - val_loss: 0.0835 - val_accuracy: 0.9737

In [23]:
# сделаем два предсказания: сначала запустим скрытые слои в одном порядке, затем - в обратном

preds1 = model(X_test, order=[0, 1]).numpy()
preds2 = model(X_test, order=[1, 0]).numpy()

assert not np.array_equal(preds1, preds2)
print(np.mean(sparse_categorical_accuracy(y_test, preds1)))
print(np.mean(sparse_categorical_accuracy(y_test, preds2)))
print(np.mean(sparse_categorical_accuracy(y_test, preds1 + preds2)))

# Можно одну нейронку исользовать как ансамбль, запуская для предсказания слои в разном направлении, потом усреднив

0.9358
0.982
0.9759


## Кастомизация цикла обучения

* Получаем на вход батч данных
* Получаем предсказание модели в режиме обучения и расчиываем loss. <br>
Эти операции мы делаем в контексте объекта tf.GradientTape(), чтобы затем получить градиенты.
* У каждой модели есть параметр .trainable_variables, в котором хранится список тензоров, являющихся обучаемыми весами модели. Мы обращаемся к этому параметру.
* Получаем значения градиентов для тензоров trainable_vars из объекта tf.GradientTape()
* Передаем веса и градиенты оптимизатору, который обновляет веса, используя градиенты.
* В последних двух строках метода train_step работаем с метриками.

In [26]:
class CustomSequential(Sequential):

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [27]:
model = CustomSequential([
    layers.InputLayer((28*28),),
    layers.Dense(784, 'relu'),
    layers.Dense(784, 'relu'),
    layers.Dense(10, 'softmax')
])

In [28]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=1024)

Epoch 1/20
59/59 [==============================] - 10s 133ms/step - loss: 0.4026 - accuracy: 0.8884 - val_loss: 0.1715 - val_accuracy: 0.9510
Epoch 2/20
59/59 [==============================] - 7s 125ms/step - loss: 0.1358 - accuracy: 0.9605 - val_loss: 0.1149 - val_accuracy: 0.9645
Epoch 3/20
59/59 [==============================] - 10s 177ms/step - loss: 0.0859 - accuracy: 0.9748 - val_loss: 0.0851 - val_accuracy: 0.9728
Epoch 4/20
59/59 [==============================] - 10s 176ms/step - loss: 0.0585 - accuracy: 0.9833 - val_loss: 0.0766 - val_accuracy: 0.9752
Epoch 5/20
59/59 [==============================] - 10s 173ms/step - loss: 0.0431 - accuracy: 0.9875 - val_loss: 0.0743 - val_accuracy: 0.9767
Epoch 6/20
59/59 [==============================] - 9s 148ms/step - loss: 0.0312 - accuracy: 0.9912 - val_loss: 0.0607 - val_accuracy: 0.9809
Epoch 7/20
59/59 [==============================] - 8s 134ms/step - loss: 0.0207 - accuracy: 0.9947 - val_loss: 0.0607 - val_accuracy: 0.9798
Ep